# Imports

In [76]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

# Neural Network
# from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV

import pickle

In [2]:
np.random.seed(66)

In [3]:
# Import Data
df = pd.read_csv('data/train-test/app_train.csv') #2015-2020 data

# Quick Details

In [4]:
df.shape

(3925778, 55)

In [6]:
df.describe()

,pitcher_id,batter_id,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,inning,inning_topbot,effective_speed,release_spin_rate,release_extension,Catcher,FirstBasemen,SecondBasemen,ThirdBasemen,ShortStop,LeftField,CenterField,RightField,at_bat_number,pitch_number,bat_score,fld_score,post_bat_score,post_fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up
count,3.925778e+06,3.925778e+06,3.910973e+06,3.902532e+06,3.902532e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.885487e+06,3.766037e+06,3.883811e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.925778e+06
mean,5.403311e+05,5.347084e+05,8.868567e+01,-7.793901e-01,5.919715e+00,5.813979e-01,7.294465e-01,8.842978e-01,8.876961e-01,-1.928299e-01,7.733834e-01,1.884104e-02,2.279551e+00,5.051610e+04,9.879194e+04,1.619614e+05,4.998506e+00,5.092058e-01,8.819325e+01,2.211626e+03,6.038816e+00,5.165356e+05,5.142169e+05,5.445410e+05,5.395915e+05,5.568228e+05,5.343705e+05,5.481872e+05,5.326239e+05,3.908339e+01,2.904070e+00,2.291132e+00,2.331207e+00,2.291209e+00,2.331207e+00,2.288409e+00,-1.288765e+02,-4.357640e+00,-2.352538e+00,2.632440e+01,-2.296656e+01,9.812009e-01
std,7.830241e+04,8.158190e+04,6.025909e+00,2.015019e+01,3.368651e+01,4.933300e-01,4.442458e-01,9.672773e-01,8.257246e-01,1.021488e+00,1.013376e+00,2.048049e+01,3.217943e+01,1.585875e+05,2.106543e+05,2.495596e+05,2.666002e+00,4.999153e-01,6.576435e+00,3.349139e+02,5.147727e-01,7.962577e+04,7.851485e+04,7.847754e+04,9.531888e+04,7.987036e+04,9.523073e+04,7.822602e+04,8.938566e+04,2.311444e+01,1.731435e+00,2.589246e+00,2.676471e+00,2.589280e+00,2.676471e+00,5.983404e+00,8.730541e+00,3.076268e+00,1.063525e+01,4.081825e+00,8.880890e+00,8.177113e-01
min,1.125260e+05,1.125260e+05,3.610000e+01,-2.521450e+04,-3.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.013900e+03,-1.600000e+01,-2.642677e+04,-2.848000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,4.130000e+02,-9.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.718084e+01,-7.721720e+02,-7.512015e+01,-1.279061e+02,-8.386989e+01,-1.843371e+02,0.000000e+00
25%,4.771320e+05,4.621010e+05,8.460000e+01,-2.130000e+00,5.630000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.000000e-01,3.000000e-01,-5.800000e-01,1.640000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,8.400000e+01,2.068000e+03,5.700000e+00,4.560780e+05,4.580150e+05,5.148880e+05,4.935960e+05,5.164160e+05,4.600750e+05,4.933160e+05,4.612350e+05,1.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.200430e+00,-1.356030e+02,-6.440000e+00,-1.126000e+01,2.331265e+01,-2.898159e+01,0.000000e+00
50%,5.436060e+05,5.434010e+05,9.000000e+01,-1.480000e+00,5.920000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.000000e-01,9.000000e-01,2.000000e-02,2.260000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,1.000000e+00,8.950000e+01,2.242000e+03,6.000000e+00,5.192220e+05,5.186920e+05,5.439390e+05,5.707990e+05,5.917200e+05,5.424550e+05,5.469910e+05,5.453410e+05,3.900000e+01,3.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,4.009364e+00,-1.307519e+02,-4.531291e+00,-3.020000e+00,2.629849e+01,-2.186600e+01,1.000000e+00
75%,6.051820e+05,5.959090e+05,9.330000e+01,9.400000e-01,6.200000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,5.000000e-01,1.330000e+00,6.100000e-01,2.870000e+00,0.000000e+00,0.000000e+00,4.551170e+05,7.000000e+00,1.000000e+00,9.310000e+01

In [7]:
df.isnull().sum()

game_date                      0
pitch_type                     0
Pitcher_name                   0
pitcher_id                     0
batter_name                    0
batter_id                      0
release_speed              14805
release_pos_x              23246
release_pos_z              23246
stand                          0
p_throws                       0
balls                          0
strikes                        0
pfx_x                      12174
pfx_z                      12174
plate_x                    12174
plate_z                    12174
on_3b                          0
on_2b                          0
on_1b                          0
inning                         0
inning_topbot                  0
effective_speed            40291
release_spin_rate         159741
release_extension          41967
Catcher                        0
FirstBasemen                   0
SecondBasemen                  0
ThirdBasemen                   0
ShortStop                      0
LeftField 

# Baseline

In [8]:
df['strike_attempt'].value_counts(normalize=True)

strike    0.459683
ball      0.359041
out       0.114194
ob        0.067082
Name: strike_attempt, dtype: float64

# Drop Nulls

---
I don't want to deal with it.

In [10]:
# df_dropped_release_speed = df[df['release_speed'].notnull()].copy()

In [12]:
# df_drop_releasepos_x = df_dropped_release_speed[df_dropped_release_speed['release_pos_x'].notnull()].copy()

In [4]:
df.dropna(subset=['release_speed','release_pos_x','release_extension','effective_speed'], inplace=True)
df.shape
# Dropped 42287 rows, which is only 1% of data, shouldn't affect 3 million points of data, may impute

(3883491, 55)

# Create `X`

In [6]:
features = ['Pitcher_name','batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up',
           'strike_attempt']
features2 = ['batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up',
            'strike_attempt']
features3 = ['pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up',
            'strike_attempt']

In [57]:
X = df[features3].sample(frac = .25, random_state=65)
a = df[features3].sample(frac = .25, random_state=66)

In [43]:
X.shape

(970873, 29)

# Dummify (if needed)

In [9]:
dummies = ['Pitcher_name','batter_name','pitch_name']

In [58]:
X_dummy = pd.get_dummies(X, columns=[col for col in dummies if col in X.columns])

In [59]:
print(X_dummy.shape)
X_dummy.head() # re-run after dropping strike_attempt

(3883491, 42)


,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,strike_attempt,pitch_name_2-Seam Fastball,pitch_name_4-Seam Fastball,pitch_name_Changeup,pitch_name_Curveball,pitch_name_Cutter,pitch_name_Eephus,pitch_name_Fastball,pitch_name_Forkball,pitch_name_Intentional Ball,pitch_name_Knuckle Curve,pitch_name_Knuckleball,pitch_name_Pitch Out,pitch_name_Screwball,pitch_name_Sinker,pitch_name_Slider,pitch_name_Split-Finger,pitch_name_Unknown
0,96.1,-1.92,5.95,1,1,1,2,-0.27,1.70,-0.93,1.62,12,0,95.4,6,2,7,3.075,-139.609,-9.724,-2.028,34.780,-11.706,2,strike,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,93.1,-1.50,6.02,1,1,1,2,0.24,0.72,0.78,1.55,12,0,92.2,5,2,7,5.156,-135.442,-7.227,5.004,28.835,-25.323,2,strike,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,97.0,-1.37,6.08,1,1,1,2,-0.50,1.41,1.32,2.47,12,0,96.4,4,2,7,8.459,-140.843,-7.313,-5.412,31.551,-15.189,2,strike,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,93.6,-1.25,6.06,1,1,0,2,0.32,0.80,2.92,2.23,12,0,92.8,3,2,7,10.094,-135.734,-5.821,6.079,26.355,-24.250,2,ball,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,97.1,-1.49,6.08,1,1,0,1,-0.76,1.63,0.26,2.74,12,0,96.0,2,2,7,6.519,-140.993,-7.138,-9.148,32.251,-11.956,2,strike,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Resample After Dummying (optional)

In [10]:
# X_sample = X_dummy.sample(frac = .5)
# print(X_sample.shape)
# X_sample.head()

(1941746, 4281)


release_speed  release_pos_x  release_pos_z  stand  p_throws  balls  \
227109            88.0           1.56           5.87      1         0      2   
3484978           93.4          -1.79           5.85      0         1      0   
2155555           82.1          -1.20           5.96      0         1      0   
1170615           95.2          -1.52           5.63      1         1      0   
818064            82.2          -2.11           4.72      1         1      0   

         strikes  pfx_x  pfx_z  plate_x  plate_z  inning  inning_topbot  \
227109         1  -0.16   0.93     1.06     2.38       3              0   
3484978        2  -0.10   1.50     0.83     2.53       5              0   
2155555        0  -0.90   0.40    -0.73     1.69       6              1   
1170615        1  -0.60   1.60    -0.34     2.84       5              0   
818064         0   0.90   0.60     0.52     1.46      12              0   

         effective_speed  release_spin_rate  pitch_number  bat_score  \
227109              85.7             1644.0             4          0   
3484978             93.7             2572.0             3          2   
2155555             82.4             1817.0             1          1   
1170615             94.5             2428.0             2          4   
818064              81.6             2726.0             1          6   

         fld_score       vx0         vy0       vz0         ax         ay  \
227109           2 -1.201000 -128.066000 -4.106000  -0.305000  27.574000   
3484978          4  6.978909 -135.832033 -6.051476  -2.695042  26.833227   
2155555          0  2.739508 -119.580807 -3.986059  -9.098241  17.716171   
1170615          4  3.565000 -138.502000 -4.539000  -6.533000  34.930000   
818064           7  2.647000 -119.677000 -1.610000  11.141000  20.800000   

                az  outs_when_up strike_attempt  Pitcher_name_A.J. Achter  \
227109  -23.718000             2           ball                         0   
3484978 -12.890802             1             ob                         0   
2155555 -28.125549             2         strike                         0   
1170615 -13.776000             2         strike                         0   
818064  -27.972000             0         strike                         0   

         Pitcher_name_A.J. Burnett  Pitcher_name_A.J. Cole  \
227109                           0                       0   
3484978                          0                       0   
2155555                          0                       0   
1170615                          0                       0   
818064                           0                       0   

         Pitcher_name_A.J. Griffin  Pitcher_name_A.J. Minter  \
227109                           0                         0   
3484978                          0                         0   
2155555                          0                         0   
1170615                          0                         0   
818064                           0                         0   

         Pitcher_name_A.J. Morris  Pitcher_name_A.J. Puk  \
227109                          0                      0   
3484978                         0                      0   
2155555                         0                      0   
1170615                         0                      0   
818064                          0                      0   

         Pitcher_name_A.J. Schugel  Pitcher_name_AJ Ramos  \
227109                           0                      0   
3484978                          0                      0   
2155555                          0                      0   
1170615                          0                      0   
818064                           0                      0   

         Pitcher_name_AJ Reed  Pitcher_name_Aaron Altherr  \
227109                      0                           0   
3484978                     0                           0   
2155555                     0                          

## Create and LabelEncode `y`

In [60]:
y = X_dummy['strike_attempt']
X_dummy.drop(columns='strike_attempt', inplace=True) # Drop after taking y

In [12]:
# y = X_sample['strike_attempt']

# X_sample.drop(columns='strike_attempt',inplace=True) # Drop the target variable in the sampled dummified X

In [61]:
X_dummy.columns[:30] # checks consistency
# X_sample.columns[:30]

Index(['release_speed', 'release_pos_x', 'release_pos_z', 'stand', 'p_throws',
       'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'inning',
       'inning_topbot', 'effective_speed', 'pitch_number', 'bat_score',
       'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up',
       'pitch_name_2-Seam Fastball', 'pitch_name_4-Seam Fastball',
       'pitch_name_Changeup', 'pitch_name_Curveball', 'pitch_name_Cutter',
       'pitch_name_Eephus'],
      dtype='object')

In [62]:
le = LabelEncoder()

In [63]:
y_encoded = le.fit_transform(y)

In [64]:
y_encoded
# le.classes_

array([3, 3, 3, ..., 3, 0, 0])

# Train Encoding for batter and pitcher

---
This is feature engineering.

In [ ]:
# Engineered Features


# Modeling

---
Skeleton for setting up model

```python
# Setting up Params
params = {[]}
# Instantitate Randomized, fuck grid
model = RandomizedSearch(REAL MODEL) # Set up to give cross validated score
model.score() # gives cv score
```

### Use X_dummy and y_encoded

## Logistic Regression

In [ ]:
# Make Sample
# DataBricks - 

# Sample that is small enough to run for laptop and large for randomness

In [65]:
# Parameters
params = {
#     'penalty':['l1','l2'],
#     'solver':['newton-cg','sag','lbfgs','saga'],
    'max_iter':[100, 150, 200],
#     'multi_class':['multinomial']
}
# Instantiate
logreg = RandomizedSearchCV(estimator=LogisticRegression(random_state=66),
                            param_distributions=params,
                            verbose=2,
                            cv=3,
                            n_jobs=7)

# Dummy then Random Sample


In [66]:
logreg.fit(X_dummy, y_encoded)

C:\Users\ncesm\anaconda3\envs\capstone\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   6 out of   9 | elapsed:  4.1min remaining:  2.1min
[Parallel(n_jobs=7)]: Done   9 out of   9 | elapsed:  7.0min finished
C:\Users\ncesm\anaconda3\envs\capstone\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomizedSearchCV(cv=3, estimator=LogisticRegression(random_state=66),
                   n_jobs=7, param_distributions={'max_iter': [100, 150, 200]},
                   verbose=2)

In [69]:
logreg.best_params_

{'max_iter': 100}

In [75]:
logreg.score(X_dummy, y_encoded)

0.516456713817542

In [13]:
params = {
#     'penalty':['l1','l2'],
#     'solver':['newton-cg','sag','lbfgs','saga'],
    'max_iter':[75, 100, 125],
#     'multi_class':['multinomial']
}
logreg2 = RandomizedSearchCV(estimator=LogisticRegression(random_state=66),
                            param_distributions=params,
                            verbose=2,
                            cv=3,
                            n_jobs=8)

In [14]:
logreg2.fit(X_dummy, y_encoded)

C:\Users\ncesm\anaconda3\envs\capstone\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed:  2.8min remaining:  3.5min
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:  4.7min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:  4.7min finished
C:\Users\ncesm\anaconda3\envs\capstone\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomizedSearchCV(cv=3, estimator=LogisticRegression(random_state=66),
                   n_jobs=8, param_distributions={'max_iter': [75, 100, 125]},
                   verbose=2)

In [16]:
logreg2.best_params_

{'max_iter': 125}

In [15]:
logreg2.score(X_dummy, y_encoded)

0.5214339366307273

# Logistic Regression, default parameters

In [10]:
lr = LogisticRegression(n_jobs=8, random_state=66)

In [11]:
lr.fit(X_dummy, y_encoded)

LogisticRegression(n_jobs=8, random_state=66)

In [12]:
lr.score(X_dummy, y_encoded) # Train Score

0.516456713817542

In [55]:
lr = LogisticRegression(n_jobs=8, random_state=66) # Sampled

In [56]:
lr.fit(X_dummy, y_encoded)

MemoryError: Unable to allocate 29.5 GiB for an array with shape (970873, 4077) and data type float64

In [12]:
lr.score(X_dummy, y_encoded) # Train Score

0.516456713817542

# SVM Model Time, default parameters

In [ ]:
svm = SVC()
svm.fit(X_dummy, y_encoded) # really robust, long time

# neural network, lots of batching